In [ ]:
!pip install pandas
!pip install pip install openai==0.28
!pip install tiktoken cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 45.0 MB/s eta 0:00:00
  Attempting uninstall: importlib_metadata
    Found existing installation: importlib-metadata 7.0.0
    Uninstalling importlib-metadata-7.0.0:
      Successfully uninstalled importlib-metadata-7.0.0


In [ ]:
import pandas as pd
import time
import openai

# Load your OpenAI API key
openai.api_key = ''

df = pd.read_excel('gia_lap_1106.xlsx')

# List to store all messages and their response times for export
all_messages = []

for index, row in df.iterrows():
    # Initialize message history and response count
    message_history = []
    response_times = []
    response_count = 0

    # Extract conversation prompts, questions, and max response count
    person_A_context = row['question_prompt']
    person_B_context = row['answer_prompt']
    current_question = row['first_question']
    max_response = row['max_response']

    # Person A starts the conversation
    message_history.append(current_question)
    response_times.append(0)  # No response time for the first question
    print(f"---\nPerson A: {current_question}")

    # Simulate conversation
    while response_count < max_response:
        # Person B responds
        start_time = time.time()
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[{"role": "system", "content": person_B_context}] + [{"role": "user", "content": msg} for msg in message_history],
            temperature=0,
            max_tokens=1024,
            top_p=1,
            frequency_penalty=1,
            presence_penalty=2
        )
        end_time = time.time()
        person_B_response = response.choices[0].message['content']
        message_history.append(person_B_response)
        response_time = end_time - start_time
        response_times.append(response_time)
        print(f"---\nPerson B: {person_B_response} (Response time: {response_time:.2f} seconds)")
        response_count += 1
        time.sleep(1)  # Sleep for 1 second

        # Check if max response count is reached
        if response_count >= max_response:
            break

        # Now Person A responds
        start_time = time.time()
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[{"role": "system", "content": person_A_context}] + [{"role": "user", "content": msg} for msg in message_history],
            temperature=0.3,
            max_tokens=1024,
            top_p=1,
            frequency_penalty=1,
            presence_penalty=2
        )
        end_time = time.time()
        current_question = response.choices[0].message['content']
        message_history.append(current_question)
        response_time = end_time - start_time
        response_times.append(response_time)
        print(f"---\nPerson A: {current_question} (Response time: {response_time:.2f} seconds)")
        time.sleep(1)  # Sleep for 1 second

    # Add conversation and response times to all messages
    for i in range(len(message_history)):
        all_messages.append([message_history[i], response_times[i]])
    all_messages.append(['---------------------------', 0])

    # Reset message history for the next conversation
    message_history = []

# Create a DataFrame and export to Excel
df_export = pd.DataFrame(all_messages, columns=['Conversation', 'Response_Time'])
df_export.to_excel('result.xlsx', index=False)

print("Conversations with response times have been exported to result.xlsx")


---
Person A: The HR in that room is a really tough woman. So you should practice with me first to ensure victory. Are you ready?
---
Person B: Yes! (Response time: 0.43 seconds)
---
Person A: 1. What is your name? (Response time: 0.49 seconds)
---
Person B: My name is [your name]. (Response time: 0.98 seconds)
---
Person A: Oh, what a lovely name you have! It's nice to meet you, [your name]. How was your day? (Response time: 0.86 seconds)
---
Person B: It was bad. (Response time: 1.04 seconds)
---
Person A: Oh, I'm sorry to hear that. Is there anything specific that made your day bad? (Response time: 0.74 seconds)
---
Person B: No, there wasn't anything specific. It was just one of those days. But thank you for asking! How about you? How was your day? (Response time: 1.41 seconds)
---
Person A: Thank you for asking, [your name]. My day has been quite productive so far. I've had the opportunity to work on some interesting projects and collaborate with my team members. Overall, it's bee

InvalidRequestError: ignored

In [ ]:
# Create a DataFrame and export to Excel
df_export = pd.DataFrame(all_messages, columns=['Conversation', 'Response_Time'])
df_export.to_excel('result.xlsx', index=False)

print("Conversations with response times have been exported to result.xlsx")

Conversations with response times have been exported to result.xlsx
